# Example of using ins-nessai

**Warning** This notebook demonstrates experimental API in nessai that may change

## Installation

We start by installing a bleeding edge version of nessai directly from GitHub.


**Warning:** this version of nessai is bleeding edge and subject to changes.

In [ ]:
!pip install git+https://github.com/mj-will/nessai.git@stable-ins --no-deps

# Initial setup

In [ ]:
from nessai.flowsampler import FlowSampler
from nessai.model import Model
from nessai.plot import corner_plot
from nessai.utils import setup_logger
import numpy as np

We then set the output directory and configure the logger.

In [ ]:
output = "ins_example"
setup_logger(output=output)

## Defining the model

As per usual we, define model which contains the log-likelihood and log-prior.

In [ ]:
class Rosenbrock(Model):
    """A Rosenbrock model"""

    def __init__(self, dims: int = 2) -> None:
        self.names = [f"x_{i}" for i in range(dims)]
        self.bounds = {n: [-5, 5] for n in self.names}

    def log_likelihood(self, x):
        """Log-likelihood function

        Parameters
        ----------
        x : numpy.ndarray
            Array of samples.

        Returns
        -------
        numpy.ndarray
            Array of log-probabilities.
        """
        # We get an unstructured view of the structured input array. This
        # allows use to vectorise the likelihood calculation.
        x = self.unstructured_view(x)
        return -np.sum(
            100. * (x[..., 1:] - x[..., :-1] ** 2.0) ** 2.0
            + (1.0 - x[..., :-1]) ** 2.0,
            axis=-1
        )

    def log_prior(self, x: np.ndarray) -> np.ndarray:
        """Log probability for a uniform prior.

        Also checks if samples are within the prior bounds.

        Parameters
        ----------
        x : numpy.ndarray
            Array of samples.

        Returns
        -------
        numpy.ndarray
            Array of log-probabilities.
        """
        # Check if the points are within the prior bounds
        log_p = np.log(self.in_bounds(x), dtype=float)
        # Compute the log-prior probability
        log_p -= np.sum(np.log(self.upper_bounds - self.lower_bounds))
        return log_p

    def to_unit_hypercube(self, x: np.ndarray) -> np.ndarray:
        """Convert the samples to the unit-hypercube.

        Parameters
        ----------
        x : numpy.ndarray
            Array of samples.

        Returns
        -------
        numpy.ndarray
            Array of rescaled samples.
        """
        x_out = x.copy()
        for n in self.names:
            x_out[n] = (
                (x[n] - self.bounds[n][0])
                / (self.bounds[n][1] - self.bounds[n][0])
            )
        return x_out

    def from_unit_hypercube(self, x: np.ndarray) -> np.ndarray:
        """Convert samples from the unit-hypercube to the prior space.

        Parameters
        ----------
        x : numpy.ndarray
            Array of samples in the unit-hypercube.

        Returns
        -------
        numpy.ndarray
            Array of sample in the prior space.
        """
        x_out = x.copy()
        for n in self.names:
            x_out[n] = (
                (self.bounds[n][1] - self.bounds[n][0])
                * x[n] + self.bounds[n][0]
            )
        return x_out

Create an instance of the model

In [ ]:
model = Rosenbrock(dims=2)

## Configuring the sampler

We run the importance nested sampler the same way we run that standard nested sampler in `nessai`, the only difference is we need to set `importance_nested_sampler=True`.

In [ ]:
sampler = FlowSampler(
    model,
    output=output,
    nlive=2000,
    importance_nested_sampler=True,  # Use the importance nested sampler
    resume=False,  # Avoid resuming
)

In [ ]:
sampler.run()

## Results

The reults are saved in the output directory, by default this includes plots as well. However, we can also visualise some of the results directly in this notebook

In [ ]:
print(
    f"Final ln-evidence: {sampler.log_evidence:.3f} +/- {sampler.log_evidence_error:.3f}"
)

In [ ]:
fig = corner_plot(sampler.posterior_samples, include=model.names)

## Diagnostics

We can also look at some of the diagnostic plots that are produced.

### Trace plot

Since the prior-volume (X) is not computed, we can't produce a traditional trace plot. However, we can plot samples against $\log (\pi / Q)$ which is refered to as `logW` in the sampler.

In [ ]:
fig = sampler.ns.plot_trace()

### State plot

We can also produce a **state** plot which that state of the sampler at each iteration. If plotting is enabled, this plot is produce during the run and can be useful for understanding the current state of the sampler.

In [ ]:
fig = sampler.ns.plot_state()